# **Neural Style Network**

## Importing Libraries

In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image
import tensorflow as tf

## Loading The Model

We will use the pretrained VGG19 model here. It has 3 components:

1.   features, that hold all the convolutional, max pool ans ReLu layers
2.   avgpool, that holds average pool layer
3.   classifier, that holds dense layers

We are going to use onyl convolutional neural network for simplicity in training. Hence are imcludong onyl features.

In [9]:
# Loading the model
model=models.vgg19(pretrained=True).features
device=torch.device( "cuda" if (torch.cuda.is_available()) else 'cpu')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Image Preprocessing

We will be using torch.tranform to

1.   resize all the images to 512 x 512
2.   to convert images into tensor

Then we will load the content and style images, and we will use a clone of content image and modify it to obtain the results. We will have to allow the clone image to be modified by gradient descent.



In [10]:
def image_loader(path):
    image = Image.open(path)
    loader=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    image=loader(image).unsqueeze(0)
    return image.to(device,torch.float)

content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

original_image=image_loader(content_path)
style_image=image_loader(style_path)
generated_image=original_image.clone().requires_grad_(True)

## Feature Representation

Defininf a class that will provide the feature representations of the intermediate layers. In this class, we will initialize a model by eliminating the unused layers of the vgg19 model and extract the activations or the feature representations of the ‘conv1_1’, ‘conv2_1’, ‘conv3_1’, ‘conv4_1’ and ‘conv5_1’ layers (index values [0, 5, 10, 19, 28]). Store these activations of 5 convolutional layers in an array and return the array.

In [11]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        self.req_features= ['0','5','10','19','28']
        self.model=models.vgg19(pretrained=True).features[:29] #model will contain the first 29 layers

    #x holds the input tensor(image) that will be feeded to each layer
    def forward(self,x):
        features=[]
        for layer_num,layer in enumerate(self.model):
            #activation of the layer will stored in x
            x=layer(x)
            #appending the activation of the selected layers and return the feature array
            if (str(layer_num) in self.req_features):
                features.append(x)

        return features

## Defining the Losses

I have discussed about the losses in detail in documentaion.

In brief, we have content and style losses, and the total loss is linear sum fo these two losses.

In [12]:
# Content loss
def calc_content_loss(gen_feat,orig_feat):
    #Calculating the content loss of each layer by calculating the MSE between the content and generated features and adding it to content loss
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

# Style loss
def calc_style_loss(gen,style):
    #Calculating the gram matrix for the style and the generated image
    batch_size,channel,height,width=gen.shape

    G=torch.mm(gen.view(channel,height*width),gen.view(channel,height*width).t())
    A=torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())

    #Calcultating the style loss of each layer by calculating the MSE between the gram matrix of the style image and the generated image and adding it to style loss
    style_l=torch.mean((G-A)**2)
    return style_l

# Total loss
def calculate_loss(gen_features, orig_feautes, style_featues):
    style_loss=content_loss=0
    for gen,cont,style in zip(gen_features,orig_feautes,style_featues):
        #extracting the dimensions from the generated image
        content_loss+=calc_content_loss(gen,cont)
        style_loss+=calc_style_loss(gen,style)

    #calculating the total loss of e th epoch
    total_loss=alpha*content_loss + beta*style_loss
    return total_loss

## Training Model

In [13]:
# Setting up hyperparameters
model=VGG().eval()

epoch=2500
lr=0.004
alpha=8
beta=70

#Using adam optimizer, it will update the generated image not the model parameter
optimizer=optim.Adam([generated_image],lr=lr)

In [ ]:
for e in range (epoch):
    gen_features=model(generated_image)
    orig_feautes=model(original_image)
    style_featues=model(style_image)

    total_loss=calculate_loss(gen_features, orig_feautes, style_featues)
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if(e/100):
        print(total_loss)

        save_image(generated_image,"gen.png")

tensor(2.9024e+10, grad_fn=<AddBackward0>)
tensor(2.6527e+10, grad_fn=<AddBackward0>)
tensor(2.4293e+10, grad_fn=<AddBackward0>)
tensor(2.2368e+10, grad_fn=<AddBackward0>)
tensor(2.0615e+10, grad_fn=<AddBackward0>)
tensor(1.8946e+10, grad_fn=<AddBackward0>)
tensor(1.7394e+10, grad_fn=<AddBackward0>)
tensor(1.5979e+10, grad_fn=<AddBackward0>)
tensor(1.4698e+10, grad_fn=<AddBackward0>)
tensor(1.3551e+10, grad_fn=<AddBackward0>)
tensor(1.2535e+10, grad_fn=<AddBackward0>)
tensor(1.1639e+10, grad_fn=<AddBackward0>)
tensor(1.0847e+10, grad_fn=<AddBackward0>)
tensor(1.0142e+10, grad_fn=<AddBackward0>)
tensor(9.5112e+09, grad_fn=<AddBackward0>)
tensor(8.9429e+09, grad_fn=<AddBackward0>)
tensor(8.4276e+09, grad_fn=<AddBackward0>)
tensor(7.9575e+09, grad_fn=<AddBackward0>)
tensor(7.5277e+09, grad_fn=<AddBackward0>)
tensor(7.1345e+09, grad_fn=<AddBackward0>)
tensor(6.7754e+09, grad_fn=<AddBackward0>)
tensor(6.4475e+09, grad_fn=<AddBackward0>)
tensor(6.1466e+09, grad_fn=<AddBackward0>)
tensor(5.86